# Java

Java is the mother of all JVM languages, first released in 1995 after years of development by Sun Microsystems.  BeakerX uses [OpenJDK](http://openjdk.java.net/) for Java and all the kernels.

BeakerX supports Java cells that define classes, which can be used by other JVM based cells.
One cell is equivalent to a Java compilation unit and can contain a single class, or a sequence of statements.

In [1]:
%classpath add mvn org.json json 20180813

In [2]:
import org.json.JSONObject;
import org.json.JSONArray;

import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;
import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Scanner;


        URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode("gare de Lyon", "UTF-8")+"&postcode=75000");
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            JSONArray res = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
    System.err.println(res);//longitude , latitude
        }


[2.3469,48.8589]


null

In [3]:
package test.beaker;

import java.util.Date;
import java.text.SimpleDateFormat;

public class BeakerTest {
  private Date _date;
  private SimpleDateFormat sdf = new SimpleDateFormat("yyyy-MM-dd'T'HH:mmZ");
  
  public BeakerTest() {
    _date = new Date();
  }

  public String getDateTxt() {
    return "Today:" + sdf.format(_date);
  }

  public String getDateUpperCaseTxt() {
    return getDateTxt().toUpperCase();
  }

}


test.beaker.BeakerTest

In [9]:
import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;

import org.json.JSONObject;
import org.json.JSONArray;

import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Map;
import java.util.HashMap;
import java.util.Scanner;

public class GeoPoint{
    public final double latitude;
    public final double longitude;
    public final Map<GeoPoint, Integer> neighborToDistance;
    
    public static List<GeoPoint> data= new ArrayList<GeoPoint>();
    
    public GeoPoint(double latitude, double longitude){
        this.latitude= latitude;
        this.longitude= longitude;
	neighborToDistance= new HashMap<GeoPoint, Integer>();
    }

    public double distanceTo(GeoPoint other){
        return distanceTo(other.latitude, other.longitude);
    }
   public double distanceTo(double latitude, double longitude){
       double earthRadius= 6371e3;// in meters
        double phi1=Math.toRadians(this.latitude);
        double phi2=Math.toRadians(latitude);
        double deltaPhi=Math.toRadians(latitude- this.latitude);
        double deltaLambda= Math.toRadians(longitude- this.longitude);
        double a= Math.sin(deltaPhi/2) * Math.sin(deltaPhi/2) +
            Math.cos(phi1) * Math.cos(phi2) * Math.sin(deltaLambda/2) * Math.sin(deltaLambda/2);
        double c= 2* Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
        return earthRadius * c;
    }
    public String toString(){
        return "GeoPoint: "+super.toString()+"{latitude: "+latitude+", longitude: " + longitude+", nb of neighbors"+neighborToDistance.size()+"}";
    }
    public static GeoPoint closerTo(double latitude, double longitude, double deltaD, Collection<GeoPoint> points){
        GeoPoint res=null;
        double minD= Double.POSITIVE_INFINITY;
        for(GeoPoint point : points){
            double currentD= point.distanceTo(latitude, longitude);
            if((currentD < minD) && (currentD > deltaD) ){
                minD= currentD;
                res= point;
            }
        }
        return res;
    }
    public static GeoPoint findByName(String name){
	    GeoPoint res= null;
        try{
            
            URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode(name, "UTF-8")+"&postcode=75000");
            try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
                JSONArray coords = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
                res= new GeoPoint(coords.getDouble(0), coords.getDouble(1));
            }catch(Exception e){
                System.err.println(e);
            }
        }catch(Exception e){
            System.err.println(e);
        }  
    return res; // should throw
    }

    public static GeoPoint closerTo(String name, Collection<GeoPoint> data){
        return closerTo(findByName(name), data);
    }
    public static GeoPoint closerTo(GeoPoint ref, Collection<GeoPoint> data){
        return closerTo(ref.latitude, ref.longitude, 1.e-50, data);
    }
public static List<GeoPoint> read(String dataUrl) throws IOException {
        List<GeoPoint> res= new ArrayList<GeoPoint>();
        URL url= new URL(dataUrl);
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            String[] headers= br.readLine().split(" ");
            int nbVertices= Integer.parseInt(headers[0]);
            int nbEdges= Integer.parseInt(headers[1]);
            for(int i=0; i != nbVertices; ++i){
                String[] coords= br.readLine().split(" ");//latitude, longitude
                res.add(new GeoPoint(Double.parseDouble(coords[0]), Double.parseDouble(coords[1])));
            }
            for(int i=0; i != nbEdges; ++i){
                String[] data= br.readLine().split(" ");//start end nbDirs Duration Length
                GeoPoint start= res.get(Integer.parseInt(data[0]));
                GeoPoint end= res.get(Integer.parseInt(data[1]));
                Integer length= Integer.parseInt(data[4]);
                start.neighborToDistance.put(end, length);
                if(data[2].equals("2")){
                    end.neighborToDistance.put(start, length);
                }
            }
        }catch (Exception e){
            System.err.println(e);
        }
        return res;
    }
    public static void main(String[] args){
        try{
            String url="https://raw.githubusercontent.com/jilljenn/tryalgo/master/examples/paris.txt";
            data= read(url);
            System.out.println(data);
        }catch(FileNotFoundException e){
            System.err.println(e);
        }
        catch(IOException e){
            System.err.println(e);
        }
    }
    
}

com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint

In [10]:
String[] args={};
GeoPoint.main(args);

[GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@187a99b1{latitude: 48.8351503, longitude: 2.3077904, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@10db9441{latitude: 48.8317022, longitude: 2.3579381, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@4e79657{latitude: 48.8812439, longitude: 2.2917141, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@5ff67148{latitude: 48.8558991, longitude: 2.3550941, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@3805fe88{latitude: 48.8406029, longitude: 2.3993259, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@6f1a29dd{latitude: 48.8587301, longitude: 2.4106012000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@fac82d6{latitude: 48.879117900000004, longitude: 2.3369039000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@67da855

 of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@6ddda369{latitude: 48.8718031, longitude: 2.2935687000000002, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@2a988c5a{latitude: 48.859518, longitude: 2.3540808, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@2d95073e{latitude: 48.8252916, longitude: 2.3626607, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@4ede130e{latitude: 48.837623300000004, longitude: 2.3223905, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@2349120b{latitude: 48.831445800000004, longitude: 2.3807413, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@75c4232e{latitude: 48.875665500000004, longitude: 2.3196156, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@4e6f30f0{latitude: 48.867839200000006, longitude: 2.3139027000000003, nb of neighbors3}, GeoPoint: com.twosigma.b

beaker.javash.bkr5c0dc8ab.GeoPoint@20c7253b{latitude: 48.8687618, longitude: 2.3110097, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@27bbfb00{latitude: 48.875008, longitude: 2.3574993, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@1bc3b665{latitude: 48.847566500000006, longitude: 2.3227914000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@25287942{latitude: 48.818424900000004, longitude: 2.3699035, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@37d972cf{latitude: 48.825801600000005, longitude: 2.371827, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@343ba049{latitude: 48.829765300000005, longitude: 2.3570651000000002, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@655c5cd7{latitude: 48.853041600000005, longitude: 2.2618851, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint

itude: 2.338355, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@15266d16{latitude: 48.888485900000006, longitude: 2.3355041, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@3a59309d{latitude: 48.8920808, longitude: 2.3462443, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@4cbdbfdf{latitude: 48.8589384, longitude: 2.4058751000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@5336755d{latitude: 48.836274700000004, longitude: 2.2570944, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@72abb757{latitude: 48.8845478, longitude: 2.3215339, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@29cad0da{latitude: 48.8450283, longitude: 2.2976957000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@22297e68{latitude: 48.848349600000006, longitude: 2.3760104, nb of neighbors3}, GeoPoint: co

sh.bkr5c0dc8ab.GeoPoint@42f1534{latitude: 48.837184400000005, longitude: 2.3001202000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@1135b179{latitude: 48.8574357, longitude: 2.3796886, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@27ac168a{latitude: 48.859144300000004, longitude: 2.3794671000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@529d617d{latitude: 48.850145000000005, longitude: 2.3483419000000003, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@6468add8{latitude: 48.8871708, longitude: 2.3495686, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@3c269219{latitude: 48.8294447, longitude: 2.3688264, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@2278d79c{latitude: 48.87276790000001, longitude: 2.397056, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@5be8f2de{l

...

7{latitude: 48.8342923, longitude: 2.2839441000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@a900978{latitude: 48.885247, longitude: 2.3033805000000003, nb of neighbors6}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@417ac363{latitude: 48.8478567, longitude: 2.4151291, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@20f6ed25{latitude: 48.8775874, longitude: 2.4082945000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@38fc7b64{latitude: 48.8481625, longitude: 2.3974807, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@a857f4c{latitude: 48.889327200000004, longitude: 2.3370177, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@29370b26{latitude: 48.8280303, longitude: 2.3924407000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@3520cd7{latitude: 48.8696671, longitude: 2.3264549000000

rs1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@5930fefb{latitude: 48.851677300000006, longitude: 2.3352858000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@5acc8ed8{latitude: 48.8535185, longitude: 2.3097623, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@58aeabe{latitude: 48.843245800000005, longitude: 2.383671, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@3e767fec{latitude: 48.8421499, longitude: 2.2816381000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@796cde3e{latitude: 48.845419, longitude: 2.2558730000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@2223ff22{latitude: 48.876739400000005, longitude: 2.2945361, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@67b055f7{latitude: 48.8869897, longitude: 2.3862934, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash

titude: 48.876027300000004, longitude: 2.3600994, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@63dea464{latitude: 48.859864800000004, longitude: 2.3517474000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@2a5e0096{latitude: 48.859753700000006, longitude: 2.3519682, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@57438a33{latitude: 48.861238500000006, longitude: 2.3606569, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@16d4ad2d{latitude: 48.878907700000006, longitude: 2.3782625, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@695862ca{latitude: 48.8859937, longitude: 2.3892805000000004, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@50cec082{latitude: 48.833441, longitude: 2.3657363, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@443ecd21{latitude: 48.865756100000006, longi

ngitude: 2.3738654, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@1940b7ef{latitude: 48.8720669, longitude: 2.3034816, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@1379d881{latitude: 48.840714500000004, longitude: 2.2682241000000003, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@569ea58d{latitude: 48.8675926, longitude: 2.3415172, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@2de3c98d{latitude: 48.852675500000004, longitude: 2.3387938000000004, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@4640aab3{latitude: 48.8585421, longitude: 2.3678391000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@4e0f1303{latitude: 48.8845286, longitude: 2.359327, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr5c0dc8ab.GeoPoint@211adcfd{latitude: 48.884465500000005, longitude: 2.3494898, nb of neighbors2}, G

null

In [12]:
double minLat= Double.MAX_VALUE;
double maxLat= -Double.MAX_VALUE;
double minLong= Double.MAX_VALUE;
double maxLong= -Double.MAX_VALUE;
for(GeoPoint p: GeoPoint.data){
    if(p.latitude < minLat){ minLat= p.latitude;}
    if(p.latitude > maxLat){ maxLat= p.latitude;}
    if(p.longitude < minLong){ minLong= p.longitude;}
    if(p.latitude > maxLong){ maxLong= p.longitude;}
}
System.out.println("lat:["+minLat+", "+maxLat+"], long:["+minLong+", "+maxLong+"]");

lat:[48.8157828, 48.901557700000005], long:[2.2519666000000003, 2.3598348000000002]


null

In [ ]:
package test.beaker;

BeakerTest bt = new BeakerTest();
return bt.getDateTxt();

In [ ]:
import java.util.List;
import java.util.ArrayList;
import com.twosigma.beakerx.chart.xychart.Plot;
import com.twosigma.beakerx.chart.xychart.plotitem.*;
import com.twosigma.beakerx.chart.Color;

Plot p = new Plot();

p.setTitle("this is a Java plot");

Bars b = new Bars();

List<Number> yList = new ArrayList<Number>();
yList.add(2);
yList.add(5);
yList.add(4);
yList.add(8);

b.setY(yList);
b.setColor(Color.blue);
b.setWidth(0.5);

p.add(b);
  
return p;

In [ ]:
package test.beaker;
interface DateGetter {
   public String getDateTxt();
}

In [ ]:
package test.beaker;
public class DG2 extends BeakerTest implements DateGetter {
}

## Using the classpath to load a jar

In [ ]:
%classpath add jar ../resources/jar/BeakerXClasspathTest.jar

In [ ]:
System.out.println("The Groovy working folder is :");
System.out.println(java.nio.file.Paths.get(".").toAbsolutePath().normalize().toString());
System.out.println("BeakerXClasspathTest.jar exists in this folder");

In [ ]:
import com.beaker.BeakerXClasspathTest;

BeakerXClasspathTest t = new BeakerXClasspathTest();
System.out.println(com.beaker.BeakerXClasspathTest.staticTest);
System.out.println(t.getObjectTest());